In [2]:
#https://www.simonlindgren.com/notes/2017/11/7/scrape-tweets-without-using-the-api
import scrapy
import pandas as pd
import glob
import json
from datetime import datetime
import time
pd.set_option('display.max_columns', 20)
pd.set_option('display.max_rows', 50)

In [3]:
files = glob.glob('TweetScraper/Data/tweet/*')
#files = glob.glob('user/*')
len(files)

2237

In [4]:
dictlist = []
for file in files:
    json_string = open(file, 'r', encoding='utf-8').read()
    json_dict = json.loads(json_string)
    dictlist.append(json_dict)

In [5]:
df_raw = pd.DataFrame(dictlist)
df_raw = df_raw.replace({'\n': ' '}, regex=True) # remove linebreaks in the dataframe
df_raw = df_raw.replace({'\t': ' '}, regex=True) # remove tabs in the dataframe
df_raw = df_raw.replace({'\r': ' '}, regex=True) # remove carriage return in the dataframe

In [6]:
df_big = pd.json_normalize(df_raw.raw_data)

In [7]:
df = df_big.drop(df_big.columns.difference(['created_at','id', 'full_text','display_text_range',
                                            'user_id','retweet_count','favorite_count']), 1)

In [8]:
df["time"] = df.created_at.apply(lambda x: datetime.strftime
                                 (datetime.strptime
                                  (x,'%a %b %d %H:%M:%S +0000 %Y'),
                                  '%Y-%m-%d %H:%M:%S'))
df = df.drop('created_at', 1)
df = df.set_index('time')
df = df.replace('\n','', regex=True)

In [9]:
search_list = ["doge","Doge", "crypto", "Crypto", "bitcoin", "Bitcoin", "Ethereum", "ethereum"]
skip = False
count = 0

for i in range(len(df.full_text)):
    for e in search_list:
        if skip == True: 
            skip = False
            continue
        if e in df.full_text.iloc[i]:
            print(e)
            print(df.index[i], ":", df.full_text.iloc[i]) 
            skip = True
            count = count + 1
            print(count)

bitcoin
2020-11-16 22:02:51 : @Maisie_Williams 🎶 Toss a bitcoin to ur Witcher 🎶
1
Doge
2020-11-17 18:45:21 : @ReflexFunds @nickidk4 @pcgamer Dojo/Doge
2
Bitcoin
2020-12-20 08:21:25 : Bitcoin is my safe word
3
Bitcoin
2020-12-20 09:24:37 : Bitcoin is almost as bs as fiat money
4
Doge
2020-12-20 09:30:04 : One word: Doge
5
Doge
2021-02-04 07:35:35 : Doge
6
Doge
2021-02-04 08:15:26 : Dogecoin is the people’s crypto
7
Doge
2021-02-04 08:27:10 : No highs, no lows, only Doge
8
Doge
2021-02-06 08:42:25 : @Grimezsz Dogecake
9
Doge
2021-02-07 02:24:19 : @Dogecoinrich 🤣🤣
10
Doge
2021-02-07 02:36:32 : @Jbuttermost @Dogecoinrich 🤣🤣
11
crypto
2021-02-07 06:24:50 : @itsALLrisky It’s the most fun crypto!
12
Doge
2021-02-07 22:25:14 : 🎶 Who let the Doge out 🎶
13
Doge
2021-02-08 05:27:36 : @itsALLrisky Doge appears to be inflationary, but is not meaningfully so (fixed # of coins per unit time), whereas BTC is arguably deflationary to a fault.Transaction speed of Doge should ideally be a few orders of m

In [51]:
# Export to csv
#df.to_csv("data.csv")

,id,full_text,display_text_range,user_id,retweet_count,favorite_count
time,,,,,,
2019-07-11 17:30:47,1149370463564357632,Motor Trend awards Tesla Model S best Car of t...,"[0, 110]",44196397,2733,24664
2020-07-18 01:59:21,1284306733720940544,If you’ve solved hard problems with phones / w...,"[0, 174]",44196397,5363,71850
2020-07-31 21:14:56,1289308589253783553,Engineering Graduates Rank Tesla &amp; SpaceX ...,"[0, 155]",1689516060,416,3612
2020-08-01 03:51:54,1289408490205614080,This BBC article provides a sensible summary f...,"[0, 88]",44196397,856,17192
2020-08-01 03:57:37,1289409927438405633,@cybrtrck Absolutely. Long-lasting art is incr...,"[10, 63]",44196397,72,1834
...,...,...,...,...,...,...
2021-05-06 19:19:34,1390385771803197454,@westcoastbill for sure!,"[15, 24]",44196397,317,8122
2021-05-06 19:23:04,1390386652007251970,@flcnhvy @westcoastbill 2024 is not out of the...,"[24, 78]",44196397,633,7190
2021-05-06 19:26:09,1390387429375369216,Public support for life on Mars is critical to...,"[0, 63]",44196397,15909,200122
